In [1]:
import numpy as np
import random
import time
import matplotlib.pyplot as plt
import matplotlib._color_data as mcd
color_list = ['salmon', 'limegreen', 'mediumturquoise', 'cornflowerblue', 'fuchsia', 'khaki']

import copy as cp
import pickle
import pandas as pd
import sys
from ipywidgets import IntProgress
from IPython.display import display


import RandomPEPS as rpeps
import StructureMatrixGenerator as smg
import trivialSimpleUpdate as tsu
import DoubleEdgeFactorGraphs as defg
import SimpleUpdate as su
import bmpslib as bmps
from datetime import date

# Comparing convergence time to fixed-point of trivial-Simple Update (tSU) and Belief Propagation (BP) over $10\times10$ random PEPS and random AFH

In [2]:
# tSU and BP parameters
N, M = 10, 10                                                  # NxM PEPS
bc = 'open'                                                   # boundary conditions
dw = 1e-6                                                     # maximal error allowed between two-body RDMS
BPU_dw = 1e-3
d = 2                                                         # tensor network physical bond dimension
bond_dimensions = [2]                                   # maximal virtual bond dimensions allowed for truncation
t_max = 1000                                                  # maximal number of BP iterations
epsilon = 1e-10                                               # convergence criteria for BP messages (not used)
dumping = 0.                                                  # BP messages dumping between [0, 1]
iterations = 1000                                             # maximal number of tSU iterations
sched = 'parallel'                                            # tSU scheduling scheme 
num_experiments = 1                                       # number of random experiments for each bond dimension
smat, _ = smg.finitePEPSobcStructureMatrixGenerator(N, M)     # generating the PEPS structure matrix
n, m = smat.shape
get_ground_state = 1                                        # flag

In [3]:
# ITE parameters
Z = np.array([[1, 0], [0, -1]])
Y = np.array([[0, -1j], [1j, 0]])
X = np.array([[0, 1], [1, 0]])
Sz = 0.5 * Z
Sy = 0.5 * Y
Sx = 0.5 * X
Opi = [Sx, Sy, Sz]
Opj = [Sx, Sy, Sz]
Op_field = np.eye(d)

In [6]:
BP_distance = []      # BP RDMS trace distance
tSU_distance = []     # tSU RDMS trace distance

## RUN

In [ ]:
for di, D_max in enumerate(bond_dimensions):
    BP_distance.append([])
    tSU_distance.append([])
    print('\n=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=')
    print('|               D = {}               |'.format(D_max))
    print('=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=')
    

    f = IntProgress(min=0, max=num_experiments, description='Runing:', bar_style='success') # instantiate the bar
    display(f) # display the bar
    for e in range(num_experiments):
        BP_distance[di].append([])
        tSU_distance[di].append([])
        f.value += 1 # signal to increment the progress bar
        
        if get_ground_state:
            ######################################### get ground state ###############################################
            # draw some random PEPS Tensor Network
            tensors, weights = smg.randomTensornetGenerator(smat, d, D_max)
            # draw some random uniform(-1, 0) ATH couplings
            interactionConstants = -np.random.rand(m)
            #interactionConstants = [-1] * m

            # calculate the BP fixed point messages and the two-body RDMs
            BPU_graph = defg.defg()
            BPU_graph = su.TNtoDEFGtransform(BPU_graph, tensors, weights, smat)
            BPU_graph.sumProduct(t_max, epsilon, 0.2, initializeMessages=1, printTime=0, RDMconvergence=0)
            BP_rdm = []
            for j in range(m):
                BP_rdm.append(tsu.BPdoubleSiteRDM1(j,
                                                   cp.deepcopy(tensors),
                                                   cp.deepcopy(weights),
                                                   smat,
                                                   cp.deepcopy(BPU_graph.messages_n2f)))

            # run BPU until convergence criteria is met
            # the convergence criteria is taken to be an upper bound over the Averaged Trace Distance (ATD) Between
            # two consecutive BPU iterations two body RDMs --- ATD < 1e-6
            dt = 0.1
            counter = 0
            while BPU_dw * dt > 1e-6:
                for i in range(iterations):
                    counter += 1
                    tensors_next, weights_next = su.simpleUpdate(tensors,
                                                                 weights,
                                                                 dt,
                                                                 interactionConstants,
                                                                 0,
                                                                 Opi,
                                                                 Opj,
                                                                 Op_field,
                                                                 smat,
                                                                 D_max,
                                                                 'BP',
                                                                 graph=BPU_graph)
                    BPU_graph.sumProduct(t_max,
                                         epsilon,
                                         0.2,
                                         initializeMessages=1,
                                         printTime=0,
                                         RDMconvergence=0)

                    ATD = 0
                    BP_rdm_next = []
                    for j in range(m):
                        BP_rdm_next.append(tsu.BPdoubleSiteRDM1(j,
                                                                cp.deepcopy(tensors_next),
                                                                cp.deepcopy(weights_next),
                                                                smat,
                                                                cp.deepcopy(BPU_graph.messages_n2f)))

                        ATD += su.traceDistance(BP_rdm[j], BP_rdm_next[j])
                    BP_rdm = BP_rdm_next
                    ATD /= m
                    print('experiment #, i, dt, ATD = {}, {}, {}, {}'.format(e, i, dt, ATD))
                    if ATD < BPU_dw * dt:
                        dt /= 2
                        BP_rdm = BP_rdm_next
                        tensors = tensors_next
                        weights = weights_next
                        break
                    tensors = tensors_next
                    weights = weights_next
            print('BPU converged in {} iterations'.format(counter))
            ground_state_energy = su.energyPerSite(tensors,
                                                   weights,
                                                   smat,
                                                   interactionConstants,
                                                   0,
                                                   Opi,
                                                   Opj,
                                                   Op_field)
            print('The ground state Energy (per site) is: {}'.format(np.round(ground_state_energy, 6)))

            ##########################################################################################################
        else:
            # draw some random PEPS Tensor Network
            tensors, weights = smg.randomTensornetGenerator(smat, d, D_max)
        
        # save TN for BP
        BP_tensors, BP_weights = cp.deepcopy(tensors), cp.deepcopy(weights)

        # constructing the dual double-edge factor graph and run a single BP iteration
        graph = defg.defg()
        graph = su.TNtoDEFGtransform(graph, BP_tensors, BP_weights, smat)
        graph.sumProduct(1, epsilon, dumping, initializeMessages=1, printTime=0, RDMconvergence=0)
        BP_rdm = []
        for j in range(m):
                BP_rdm.append(tsu.BPdoubleSiteRDM1(j, BP_tensors, BP_weights, smat, cp.deepcopy(graph.messages_n2f)))
        print('\nBP:')
        # run BP and calculate two body rdms between two consecutive BP iterations
        for t in range(t_max):
            graph.sumProduct(1, epsilon, dumping, initializeMessages=1, printTime=0, RDMconvergence=0)

            ATD_BP = 0
            BP_rdm_next = []
            for j in range(m):
                BP_rdm_next.append(tsu.BPdoubleSiteRDM1(j,
                                                        BP_tensors,
                                                        BP_weights,
                                                        smat,
                                                        cp.deepcopy(graph.messages_n2f)))

                ATD_BP += tsu.traceDistance(BP_rdm_next[j], BP_rdm[j])
                BP_rdm[j] = BP_rdm_next[j]
            ATD_BP /= m
            BP_distance[di][e].append(ATD_BP)
            print('The ATD_BP is: {} at iteration {}'.format(ATD_BP, t))
            if ATD_BP < dw:
                #print('\n')
                #print('The final ATD_BP is: {} at iteration {}'.format(ATD_BP, t + 1))
                break

        # calculate the double site rdm in tsu
        tSU_rdm = []
        for i in range(m):
            tSU_rdm.append(tsu.doubleSiteRDM(i, tensors, weights, smat))   

        print('\ntSU:')
        # trivial SU run
        for i in range(iterations):
            tensors_next, weights_next = tsu.trivialsimpleUpdate(tensors,
                                                                 weights,
                                                                 smat,
                                                                 D_max, 
                                                                 scheduling='parallel')
            ATD = 0
            tSU_rdm_next = []
            for j in range(m):
                tSU_rdm_next.append(tsu.doubleSiteRDM(j, tensors_next, weights_next, smat))
                ATD += tsu.traceDistance(tSU_rdm_next[j], tSU_rdm[j])
                tSU_rdm[j] = tSU_rdm_next[j]
            ATD /= m
            tSU_distance[di][e].append(ATD)
            print('The ATD_BP is: {} at iteration {}'.format(ATD, i))
            if ATD < dw:
                #print('The ATD is: {} at iteration {}'.format(ATD, i))
                tensors = tensors_next
                weights = weights_next
                break
            tensors = tensors_next
            weights = weights_next  


        # calculate Averaged Trace Distance between the BP and tSU rdms.
        ATD_BP_tSU = 0
        for i in range(m):
            ATD_BP_tSU += tsu.traceDistance(BP_rdm[i], tSU_rdm[i])
        ATD_BP_tSU /= m
        print('the total ATD between BP and tSU is {}'.format(ATD_BP_tSU))
        



=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
|               D = 2               |
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=


IntProgress(value=0, bar_style='success', description='Runing:', max=1)

experiment #, i, dt, ATD = 0, 0, 0.1, 0.023479949058481484
experiment #, i, dt, ATD = 0, 1, 0.1, 0.025398289534740482
experiment #, i, dt, ATD = 0, 2, 0.1, 0.02615036076231116
experiment #, i, dt, ATD = 0, 3, 0.1, 0.026788531535878537
experiment #, i, dt, ATD = 0, 4, 0.1, 0.027776403904277148
experiment #, i, dt, ATD = 0, 5, 0.1, 0.028765369752247836
experiment #, i, dt, ATD = 0, 6, 0.1, 0.029981297951389596
experiment #, i, dt, ATD = 0, 7, 0.1, 0.03140134806511658
experiment #, i, dt, ATD = 0, 8, 0.1, 0.0331035772127445
experiment #, i, dt, ATD = 0, 9, 0.1, 0.035043593524728724
experiment #, i, dt, ATD = 0, 10, 0.1, 0.03738085148707429
experiment #, i, dt, ATD = 0, 11, 0.1, 0.040059724343085065
experiment #, i, dt, ATD = 0, 12, 0.1, 0.0429870419021918
experiment #, i, dt, ATD = 0, 13, 0.1, 0.04640531490624125
experiment #, i, dt, ATD = 0, 14, 0.1, 0.04999190325078559
experiment #, i, dt, ATD = 0, 15, 0.1, 0.054007661506494664
experiment #, i, dt, ATD = 0, 16, 0.1, 0.05851175494756109


## PLOT RESULTS

In [ ]:
name = str(N) + 'x' + str(M) + ' random PEPS'
fonts = 20
names = []
for di, D in enumerate(bond_dimensions):
    plt.figure(figsize=(12, 8))
    for e in range(num_experiments):
        plt.scatter(range(1, len(BP_distance[di][e]) + 1),
                    np.log10(np.asarray(BP_distance[di][e])),
                    color=mcd.CSS4_COLORS[color_list[di]], 
                    s=50)
        plt.scatter(range(1, len(tSU_distance[di][e]) + 1),
                    np.log10(np.asarray(tSU_distance[di][e])),
                    color=mcd.CSS4_COLORS[color_list[di + 1]], 
                    s=50)
    
    #names.append('D = ' + str(D))
    plt.title(name, fontsize=fonts)
    plt.xlabel('# iterations', fontsize=fonts)
    plt.ylabel('ATD', fontsize=fonts)
    #plt.xticks(list(range(1, num_experiments + 1, 2)))
    plt.legend(['BP', 'tSU'], fontsize=fonts)
    plt.grid()
    plt.show()
#plt.savefig(name + '.svg', format="svg")
#plt.savefig(name + '.pdf')

In [ ]:
with open("test.txt", "w") as text_file:
    for e in range(1, num_experiments + 1):
        text_file.write("\n")
        text_file.write("Experiment {}:\n".format(e))
        text_file.write("BP:\n {}\n".format(BP_distance[0][e - 1]))
        text_file.write("tSU:\n {}\n".format(tSU_distance[0][e - 1]))